# Graph

In [ ]:
%cd -q ..

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import uuid

import networkx as nx
import weldx
from weldx.asdf.tags.weldx.core.graph import (
    MeasurementChain,
    SignalClass,
    SignalTransform,
)

## Minimal example

In [ ]:
# minimal example
G = nx.DiGraph()
G.add_nodes_from(("A", "B", "C"))
G.add_edges_from([("A", "B"), ("C", "B")])
buff = weldx.asdf.util.write_buffer(dict(graph=G))
weldx.asdf.util.notebook_fileprinter(buff)

In [ ]:
weldx.asdf.util.asdf_json_repr(buff)

## cycle example

In [ ]:
# minimal example
G = nx.DiGraph()
G.add_nodes_from(("A", "B", "C", "D"))
G.add_edges_from([("B", "C"), ("A", "B"), ("C", "D"), ("D", "B")])
buff = weldx.asdf.util.write_buffer(dict(graph=G))
nx.draw(G, with_labels=True)
weldx.asdf.util.notebook_fileprinter(buff)

In [ ]:
tree = weldx.asdf.util.read_buffer(buff)
G2 = tree["graph"]
nx.draw(G2, with_labels=True)

### complex example

In [ ]:
rid = uuid.uuid4()

G = nx.DiGraph()
G.add_edges_from(
    [("A", "B"), ("A", "C"), ("A", "F"), ("D", "C"), ("B", "H"), ("X", "A"), (rid, "D")]
)
nx.draw(G, with_labels=True)
buff = weldx.asdf.util.write_buffer(dict(graph=G))
weldx.asdf.util.asdf_json_repr(buff, ("graph",))

## Measurement Example

In [ ]:
rid = uuid.uuid4()

G = nx.DiGraph()
G.add_edges_from([("A", "B"), ("B", "C"), ("C", "D")])
nx.draw(G, with_labels=True)

In [ ]:
signal = SignalClass("The Signal", 12.3)
nx.set_node_attributes(G, signal, "signal")

In [ ]:
signal_transform = SignalTransform(func="a*b + c")
nx.set_edge_attributes(G, signal_transform, "signal_transform")

In [ ]:
m = MeasurementChain(graph=G)

In [ ]:
buff = weldx.asdf.util.write_buffer(dict(measurement_chain=m))
weldx.asdf.util.asdf_json_repr(buff, ("measurement_chain",))

In [ ]:
weldx.asdf.util.notebook_fileprinter(buff)

In [ ]:
tree = weldx.asdf.util.read_buffer(buff)
G2 = tree["measurement_chain"].graph

In [ ]:
assert all([e in G.edges for e in G2.edges])
assert all([n in G.nodes for n in G2.nodes])

In [ ]:
for node in G:
    assert G.nodes[node] == G2.nodes[node]

In [ ]:
for edge in G.edges:
    assert G.edges[edge] == G2.edges[edge]